In [24]:
from models.waffleiron.segmenter import Segmenter
import torch
from datasets import LIST_DATASETS, Collate
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from collections import OrderedDict
import warnings
import copy
import random
import numpy as np
import os

import argparse
import wandb
from torchmetrics.classification import MulticlassJaccardIndex
import torchmetrics
from tqdm import tqdm
from sklearn.metrics import confusion_matrix

import torchhd
from torchhd.models import Centroid
from torchhd import embeddings

import pandas as pd

In [25]:
class Feature_Extractor:
    def __init__(self, input_channels=5, feat_channels=768, depth=48, 
                 grid_shape=[[256, 256], [256, 32], [256, 32]], nb_class=16, layer_norm=True, 
                 device=torch.device("cpu"), early_exit = 48, **kwargs):
        self.model = Segmenter(
            input_channels=input_channels,
            feat_channels=feat_channels,
            depth=depth,
            grid_shape=grid_shape,
            nb_class=nb_class, # class for prediction
            #drop_path_prob=config["waffleiron"]["drop_path"],
            layer_norm=layer_norm,
        )

        classif = torch.nn.Conv1d(
            feat_channels, nb_class, 1 # So it fits 16 = nb_class but classifier is not used
        )
        torch.nn.init.constant_(classif.bias, 0)
        torch.nn.init.constant_(classif.weight, 0)
        self.model.classif = torch.nn.Sequential(
            torch.nn.BatchNorm1d(feat_channels),
            classif,
        )

        for p in self.model.parameters():
            p.requires_grad = False
        for p in self.model.classif.parameters():
            p.requires_grad = True

        def get_optimizer(parameters):
            return torch.optim.AdamW(
                parameters,
                lr=0.001,
                weight_decay=0.003,
            )

        optim = get_optimizer(self.model.parameters())
        self.device = device
        self.device_string = "cuda:0"
        self.num_classes = nb_class
        self.early_exit = early_exit
        self.kwargs = kwargs
    
    def load_pretrained(self, path):
        # Load pretrained model
        path_to_ckpt = path
        checkpoint = torch.load(path_to_ckpt,
            map_location=self.device_string)
        state_dict = checkpoint["net"]  # Adjust key as needed
        new_state_dict = OrderedDict()

        for k, v in state_dict.items():
            new_key = k.replace("module.", "")  # Remove "module." prefix
            new_state_dict[new_key] = v

        self.model.load_state_dict(new_state_dict)

        print(
            f"Checkpoint loaded on {self.device_string}: {path_to_ckpt}"
        )

        if self.device_string != 'cpu':
            torch.cuda.set_device(self.device_string) # cuda:0
            self.model = self.model.cuda(self.device_string) # cuda:0

        self.model.eval()

    def forward_model(self, it, batch):
        pts_og = batch["points"]
        labels_og = batch["labels"]
        pts_voxel = batch["points_voxel"]
        feat = batch["feat"]
        labels = batch["labels_orig"]
        cell_ind = batch["cell_ind"]
        occupied_cell = batch["occupied_cells"]
        neighbors_emb = batch["neighbors_emb"]
        if self.device_string != 'cpu':
            feat = feat.cuda(0, non_blocking=True)
            labels = labels.cuda(0, non_blocking=True)
            batch["upsample"] = [
                up.cuda(0, non_blocking=True) for up in batch["upsample"]
            ]
            cell_ind = cell_ind.cuda(0, non_blocking=True)
            occupied_cell = occupied_cell.cuda(0, non_blocking=True)
            neighbors_emb = neighbors_emb.cuda(0, non_blocking=True)
        net_inputs = (feat, cell_ind, occupied_cell, neighbors_emb)

        if self.device_string != 'cpu':
            with torch.autocast("cuda", enabled=True):
                # Logits
                with torch.no_grad():
                    out = self.model(*net_inputs, self.early_exit)
                    encode, tokens, out = out[0], out[1], out[2]
                    pred_label = out.max(1)[1]

                    # Only return samples that are not noise
                    #torch.cuda.synchronize(device=self.device)
                    where = labels != 255
                    #torch.cuda.synchronize(device=self.device)
        else:
            with torch.no_grad():
                out = self.model(*net_inputs, self.early_exit)
                encode, tokens, out = out[0], out[1], out[2]
                pred_label = out.max(1)[1]

                # Only return samples that are not noise
                where = labels != 255

        pts_og = torch.tensor(pts_og[0]).to(self.device)
        pts_voxel = torch.tensor(pts_voxel[0]).to(self.device)
        where = where.cpu()
        return pts_og, labels_og, pts_voxel[where], tokens[0,:,where], labels[where], pred_label[0, where]

    def test(self, loader, stop):        
        # Metric
        miou = MulticlassJaccardIndex(num_classes=self.num_classes, average=None).to(self.device, non_blocking=True)
        final_labels_sep = torch.empty((1, 90000), device=self.device)
        final_labels_vox_sep = torch.empty((1, 90000), device=self.device)
        final_feat_sep = torch.empty((1, 768, 90000), device=self.device)
        final_soa_result_sep = torch.empty((1, 90000), device=self.device)
        final_points_og_sep = torch.empty((1, 3, 90000), device=self.device)
        final_points_sep = torch.empty((1, 3, 90000), device=self.device)
        num_voxels = []
        
        start_idx = 0
        for it, batch in tqdm(enumerate(loader), desc="SoA testing"):
            pts_og, labels_og, pts, features, labels, soa_result = self.forward_model(it, batch)

            # Print og labels and points
            pts_og = torch.transpose(pts_og, 0, 1)
            points_here = torch.cat((pts_og[:3, :], torch.reshape(torch.tensor((labels_og)), (1, pts_og.shape[1]))), dim=0)
            points_here = torch.transpose(points_here, 0, 1)
            points_here[points_here[:, 3] == 255, 3] = -1 # SemanticKitti
            print(points_here.shape)
            
            # Convert to NumPy
            tensor_np = points_here.cpu().numpy()
            
            # Save as CSV
            df = pd.DataFrame(tensor_np)
            df.to_csv(f"tensor_semkitti_{it}_og.csv", index=False, header=False)
            shape_sample = labels.shape[0]
            num_voxels.append(shape_sample)
            labels_tensor = torch.reshape(torch.Tensor(labels), (1,shape_sample)).to(self.device)
            feat_tensor = torch.reshape(torch.Tensor(features), (1,768,shape_sample)).to(self.device)
            soa_tensor = torch.reshape(torch.Tensor(soa_result), (1,shape_sample)).to(self.device)
            pts_tensor = torch.reshape(torch.Tensor(pts), (1,3,shape_sample)).to(self.device)
            final_labels_sep = torch.concat((final_labels_sep, F.pad(input=labels_tensor, pad=(0, 90000 - shape_sample), mode='constant', value=0)))
            final_soa_result_sep = torch.concat((final_soa_result_sep, F.pad(input=soa_tensor, pad=(0, 90000 - shape_sample), mode='constant', value=0)))
            final_feat_sep = torch.concat((final_feat_sep, F.pad(input=feat_tensor, pad=(0, 90000 - shape_sample), mode='constant', value=0)))
            final_points_sep = torch.concat((final_points_sep, F.pad(input=pts_tensor, pad=(0, 90000 - shape_sample), mode='constant', value=0)))
            print(final_labels_sep.shape)
            print(final_soa_result_sep.shape)
            print(final_feat_sep.shape)
            print(final_points_sep.shape)

            #labels = labels.to(dtype = torch.int64, device = self.device, non_blocking=True)
            #soa_result = soa_result.to(device=self.device, non_blocking=True)
            #final_labels[start_idx:start_idx+shape_sample] = labels

            #final_pred[start_idx:start_idx+shape_sample] = soa_result

            #start_idx += shape_sample

            if it == stop:
                break

        #final_labels = final_labels[:start_idx]
        #final_pred = final_pred[:start_idx]

        #print("================================")

        #print('Pred FE', final_pred, "\tShape: ", final_pred.shape)
        #print('Label', final_labels, "\tShape: ", final_labels.shape)
        #accuracy = miou(final_pred, final_labels)
        #avg_acc = torch.mean(accuracy)
        #print(f'accuracy: {accuracy}')
        #print(f'avg acc: {avg_acc}')

        #cm = confusion_matrix(pred_hd, first_label, labels=torch.Tensor(range(0,15)))
        #print("Confusion matrix \n")
        #print(cm)

        #print("================================")

        return final_points_sep, final_labels_sep, final_soa_result_sep, final_feat_sep, num_voxels

In [26]:
fe = Feature_Extractor(nb_class=19) # 19
fe.load_pretrained('/root/main/ScaLR/saved_models/ckpt_last_kitti.pth') #  ckpt_last_scalr

/tmp/ipykernel_183589/1083699130.py:47: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(path_to_ckpt,


Checkpoint loaded on cuda:0: /root/main/ScaLR/saved_models/ckpt_last_kitti.pth


In [27]:
kwargs = { # Nuscenes and semantickitti
    "rootdir": '/root/main/dataset/semantickitti', # nuscenes
    "input_feat": ["intensity", "xyz", "radius"],
    "voxel_size": 0.1,
    "num_neighbors": 16,
    "dim_proj": [2, 1, 0],
    "grids_shape": [[256, 256], [256, 32], [256, 32]],
    "fov_xyz": [[-64, -64, -8], [64, 64, 8]], # Check here
}

### FALSE NOT THE TRUE VALUES

#kwargs = {
#    "rootdir": '/root/main/dataset/semantickitti',
#    "input_feat": ["intensity", "xyz", "radius"],
#    "voxel_size": 0.1,
#    "num_neighbors": 16,
#    "dim_proj": [2, 1, 0],
#    "grids_shape": [[250, 250], [250, 12], [250, 12]],
#    "fov_xyz": [[-50, -50, -3], [50, 50, 2]], # Check here
#}

# Get datatset
DATASET = LIST_DATASETS.get('semantic_kitti') #  nuscenes

#### SemanticKitti ####

dataset_train = DATASET(
    phase="train",
    **kwargs,
)

# Validation dataset
dataset_val = DATASET(
    phase="val",
    **kwargs,
)


##### Nuscenes ####
# Train dataset
"""
dataset = DATASET(
    phase="train",
    **kwargs,
)

dataset_train = copy.deepcopy(dataset)
dataset_val = copy.deepcopy(dataset)
dataset_train.init_training()
dataset_val.init_val()
"""

num_classes = 19 # 16

path_pretrained = '/root/main/ScaLR/saved_models/ckpt_last_scalr.pth' # ckpt_last_kitti

Using original split
Using original split


In [28]:
device = torch.device("cuda")
print("Using {} device".format(device))

train_loader = torch.utils.data.DataLoader(
        dataset_train,
        batch_size=1,
        pin_memory=True,
        drop_last=True,
        collate_fn=Collate(device=device),
        persistent_workers=False,
    )

val_loader = torch.utils.data.DataLoader(
    dataset_val,
    batch_size=1,
    pin_memory=True,
    drop_last=True,
    collate_fn=Collate(device=device),
    persistent_workers=False,
)

Using cuda device


In [16]:
pts_voxel, labels_train, soa_train, feat_train, vox_train = fe.test(train_loader, 10) # With the nuscenes hyperparameters...

SoA testing: 0it [00:00, ?it/s]

torch.Size([34688, 4])


SoA testing: 1it [00:02,  2.47s/it]

torch.Size([2, 90000])
torch.Size([2, 90000])
torch.Size([2, 768, 90000])
torch.Size([2, 3, 90000])
torch.Size([34688, 4])


SoA testing: 2it [00:04,  2.42s/it]

...

torch.Size([10, 90000])
torch.Size([10, 90000])
torch.Size([10, 768, 90000])
torch.Size([10, 3, 90000])
torch.Size([34720, 4])


SoA testing: 10it [00:31,  3.41s/it]

torch.Size([11, 90000])
torch.Size([11, 90000])
torch.Size([11, 768, 90000])
torch.Size([11, 3, 90000])
torch.Size([34688, 4])


SoA testing: 10it [00:35,  3.58s/it]

torch.Size([12, 90000])
torch.Size([12, 90000])
torch.Size([12, 768, 90000])
torch.Size([12, 3, 90000])


In [17]:
torch.tensor(vox_train)

tensor([16247, 12222, 14237, 17581, 17113, 14250, 15004, 16066, 16474, 14231,
        16595])

In [18]:
torch.save(labels_train[1:], 'labels_train_nuscenes.pt')
torch.save(soa_train[1:], 'soa_train_nuscenes.pt')
torch.save(feat_train[1:], 'feat_train_nuscenes.pt')
torch.save(pts_voxel[1:], 'pts_train_nuscenes.pt')

In [19]:
torch.save(torch.tensor(vox_train), 'voxels_train_nuscenes.pt')

In [11]:
pts, labels, soa, feat, vox = fe.test(val_loader, 50) # With the nuscenes hyperparameters...

SoA testing: 1it [00:03,  3.68s/it]

torch.Size([2, 90000])
torch.Size([2, 90000])
torch.Size([2, 768, 90000])
torch.Size([2, 3, 90000])


SoA testing: 2it [00:07,  3.78s/it]

torch.Size([3, 90000])
torch.Size([3, 90000])
torch.Size([3, 768, 90000])
torch.Size([3, 3, 90000])


SoA testing: 3it [00:11,  3.94s/it]

...

torch.Size([50, 90000])
torch.Size([50, 90000])
torch.Size([50, 768, 90000])
torch.Size([50, 3, 90000])


SoA testing: 50it [06:52, 12.90s/it]

torch.Size([51, 90000])
torch.Size([51, 90000])
torch.Size([51, 768, 90000])
torch.Size([51, 3, 90000])


SoA testing: 50it [07:04,  8.49s/it]

torch.Size([52, 90000])
torch.Size([52, 90000])
torch.Size([52, 768, 90000])
torch.Size([52, 3, 90000])


In [12]:
torch.save(labels[1:], 'labels_test_semkitti.pt')
torch.save(soa[1:], 'soa_test_semkitti.pt')
torch.save(feat[1:], 'feat_test_semkitti.pt')
torch.save(torch.tensor(vox), 'voxels_test_semkitti.pt')
torch.save(pts[1:], 'pts_test_semkitti.pt')

# Check if the two models have the same weights?

In [71]:
# SemanticKitti
fe.model

Segmenter(
  (embed): Embedding(
    (norm): BatchNorm1d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv1): Conv1d(5, 768, kernel_size=(1,), stride=(1,))
    (conv2): Sequential(
      (0): BatchNorm2d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Conv2d(5, 768, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (2): BatchNorm2d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU(inplace=True)
      (4): Conv2d(768, 768, kernel_size=(1, 1), stride=(1, 1), bias=False)
    )
    (final): Conv1d(1536, 768, kernel_size=(1,), stride=(1,))
  )
  (waffleiron): WaffleIron(
    (channel_mix): ModuleList(
      (0-47): 48 x ChannelMix(
        (norm): myLayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): Sequential(
          (0): Conv1d(768, 768, kernel_size=(1,), stride=(1,))
          (1): ReLU(inplace=True)
          (2): Conv1d(768, 768, kernel_size=(1,), stride=(1,))
        )
 

In [53]:
fe_2 = Feature_Extractor(nb_class=16)
fe_2.load_pretrained('/root/main/ScaLR/saved_models/ckpt_last_scalr.pth')

/tmp/ipykernel_15601/2784029278.py:47: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(path_to_ckpt,


Checkpoint loaded on cuda:0: /root/main/ScaLR/saved_models/ckpt_last_scalr.pth


In [72]:
import torch
from torch.nn import Module

# Assuming `model1` and `model2` are your two models
def compare_model_weights(model1: Module, model2: Module):
    for (name1, param1), (name2, param2) in zip(model1.named_parameters(), model2.named_parameters()):
        if name1 != name2:
            print(f"Parameter names differ: {name1} vs {name2}")
            return False
        if not torch.equal(param1.data, param2.data):
            print(f"Weights differ in {name1}")
            return False
    print("All weights are didentical!")
    return True

# Call the function with your two models
result = compare_model_weights(fe.model, fe_2.model)

Weights differ in classif.0.weight


In [96]:
x = torch.ones((5,))
F.pad(input=x, pad=(0, 5), mode='constant', value=0)

tensor([1., 1., 1., 1., 1., 0., 0., 0., 0., 0.])

# See nuscenes number of voxels

In [2]:
import numpy as np
voxels = np.load('/root/main/ScaLR/debug/nuscenes/num_voxels.npy')

In [3]:
voxels

array([18197., 16838., 18008., 17197., 17114., 16733., 17014., 16558.,
       16587., 14200., 17857.], dtype=float32)

In [20]:
points = torch.load('pts_train_nuscenes.pt', weights_only="False")
voxels = torch.load('voxels_train_nuscenes.pt', weights_only="False")
labels = torch.load('labels_train_nuscenes.pt', weights_only="False")

In [21]:
import pandas as pd

for i in range(11):
    points_here = points[i]
    voxels_here = voxels[i]
    points_here = points_here[:, :voxels_here]
    labels_here = torch.reshape(labels[i][:voxels_here], (1, points_here.shape[1]))
    points_here = torch.cat((points_here, labels_here), dim=0)
    points_here = torch.transpose(points_here, 0, 1)
    print(points_here.shape)
    # Convert to NumPy
    tensor_np = points_here.cpu().numpy()
    
    # Save as CSV
    df = pd.DataFrame(tensor_np)
    df.to_csv(f"tensor_nuscenes_{i}.csv", index=False, header=False)

torch.Size([16247, 4])
torch.Size([12222, 4])
torch.Size([14237, 4])
torch.Size([17581, 4])
torch.Size([17113, 4])
torch.Size([14250, 4])
torch.Size([15004, 4])
torch.Size([16066, 4])
torch.Size([16474, 4])
torch.Size([14231, 4])
torch.Size([16595, 4])


# Retrain Clasification layer

In [40]:
import torch
import torch.nn as nn
import torch.optim as optim

# Example feature size (feat_channels) and number of classes
feat_channels = 768
num_classes = 19

# Define classification layer
classif = nn.Linear(feat_channels, num_classes)

# Define model using Sequential
model = nn.Sequential(
    nn.BatchNorm1d(feat_channels),
    classif
)

# Move to device (if using GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Sequential(
  (0): BatchNorm1d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (1): Linear(in_features=768, out_features=19, bias=True)
)

In [41]:
# CrossEntropyLoss for classification
criterion = nn.CrossEntropyLoss()

# Use Adam optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [47]:
num_epochs = 10  # Set number of epochs

for epoch in range(num_epochs):
    model.train()  # Set model to training mode

    optimizer.zero_grad()  # Clear previous gradients

    for it, batch in tqdm(enumerate(train_loader), desc="SoA testing"):
        pts_og, labels_og, pts, features, labels, soa_result = fe.forward_model(it, batch)
        features = torch.transpose(features, 0, 1).to(torch.float32)
        print(features.shape)
        print(labels.shape)

        outputs = model(features)  # Forward pass
        loss = criterion(outputs, labels.to(torch.int64))  # Compute loss
    
        loss.backward()  # Backpropagation
        optimizer.step()  # Update weights

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

SoA testing: 1it [00:01,  1.53s/it]

torch.Size([48422, 768])
torch.Size([48422])


SoA testing: 2it [00:02,  1.24s/it]

torch.Size([33621, 768])
torch.Size([33621])


SoA testing: 3it [00:04,  1.44s/it]

...

torch.Size([43992, 768])
torch.Size([43992])


SoA testing: 599it [32:25,  3.44s/it]

torch.Size([56359, 768])
torch.Size([56359])


SoA testing: 600it [32:28,  3.25s/it]

torch.Size([46673, 768])
torch.Size([46673])
Epoch [10/10], Loss: 2.1821


 Epoch [1/10] - 3.7914
 Epoch [2/10] - 3.3096
 Epoch [3/10], Loss: 6.6253
 Epoch [4/10], Loss: 5.0225
 Epoch [5/10], Loss: 7.8791 :(
 Epoch [6/10], Loss: 3.9678
 Epoch [7/10], Loss: 5.1117
 Epoch [8/10], Loss: 3.2974
 Epoch [9/10], Loss: 1.7246
 Epoch [10/10], Loss: 2.1821

In [52]:
# Metric
miou = MulticlassJaccardIndex(num_classes=19, average=None).to(torch.device('cuda'))

final_shape = 0

for it, batch in tqdm(enumerate(train_loader)):
    pts_og, labels_og, pts, features, labels, soa_result = fe.forward_model(it, batch)
    final_shape += features.shape[1]
    if it == 100:
        break
    

100it [05:35,  3.36s/it]


In [66]:
final_labels = torch.empty((final_shape), device=torch.device('cuda'))
final_pred = torch.empty((final_shape), device=torch.device('cuda'))
model.eval()  # Set model to training mode

start_idx = 0
for it, batch in tqdm(enumerate(train_loader)):
    pts_og, labels_og, pts, features, labels, soa_result = fe.forward_model(it, batch)
    features = torch.transpose(features, 0, 1).to(torch.float32)
    shape_here = int(features.shape[0])
    outputs = model(features)  # Forward pass
    print(outputs)
    outputs = torch.argmax(features, dim=1)
    print(outputs)
    final_labels[start_idx:start_idx+shape_here] = labels
    final_pred[start_idx:start_idx+shape_here] = outputs
    
    start_idx += shape_here

    if it == 100:
        break

1it [00:01,  1.50s/it]

tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
tensor([105, 105, 105,  ..., 105, 105, 105], device='cuda:0')


2it [00:02,  1.33s/it]

tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
tensor([105, 105, 105,  ..., 105, 105, 105], device='cuda:0')


3it [00:04,  1.45s/it]

tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
tensor([613, 664, 613,  ..., 105, 613, 105], device='cuda:0')


4it [00:06,  1.62s/it]

tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
tensor([105, 105, 105,  ..., 105, 105, 105], device='cuda:0')


4it [00:06,  1.68s/it]


KeyboardInterrupt: 

In [63]:
print("================================")

#print('pred_ts', pred_ts)
print('label', final_labels, "\tShape: ", final_labels.shape)
print('pred_hd', final_pred, "\tShape: ", final_pred.shape)
accuracy = miou(final_pred, final_labels)
avg_acc = torch.mean(accuracy)
print(f'accuracy: {accuracy}')
print(f'avg acc: {avg_acc}')

label tensor([14., 14., 13.,  ..., 14., 14., 14.], device='cuda:0') 	Shape:  torch.Size([5213235])
pred_hd tensor([105., 105., 105.,  ..., 105., 105., 105.], device='cuda:0') 	Shape:  torch.Size([5213235])


RuntimeError: shape '[19, 19]' is invalid for input of size 1109